# Preparing data

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
height = 252
width = 252
batch_size = 64
datagen = ImageDataGenerator(rescale=1./ 255,samplewise_center=True, samplewise_std_normalization=True, validation_split=0.1)
train_generator = datagen.flow_from_directory(
        'BOSSbase_1.01-256-convolved',  # this is the target directory
        batch_size=batch_size,
        class_mode='categorical',
        subset='training',
        color_mode='grayscale',
        target_size = (height, width))

validation_generator = datagen.flow_from_directory(
        'BOSSbase_1.01-256-convolved',  # this is the target directory
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation',
        color_mode='grayscale',
        target_size = (height, width))

# GNCNN Model

In [ ]:
import keras
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, AveragePooling2D, Activation
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.utils.generic_utils import get_custom_objects
from livelossplot import PlotLossesKeras

In [ ]:
num_classes = 2

In [ ]:
def gaussian_activation(x):
    alpha = 1
    sigma = 0.1
    return alpha*K.exp(-(x*x)/(sigma*sigma))

get_custom_objects().update({'gaussian_activation': Activation(gaussian_activation)})

In [ ]:
model = Sequential()
model.add(Conv2D(16, 5, activation='gaussian_activation', padding='same', input_shape=(height, width, 1)))
model.add(AveragePooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Conv2D(16, 3, activation='gaussian_activation', padding='valid'))
model.add(AveragePooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Conv2D(16, 3, activation='gaussian_activation', padding='valid'))
model.add(AveragePooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Conv2D(16, 3, activation='gaussian_activation', padding='valid'))
model.add(AveragePooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Conv2D(16, 5, activation='gaussian_activation', padding='valid'))
model.add(AveragePooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
sgd = keras.optimizers.SGD(lr=0.005, momentum=0.0, decay=0.0000005, nesterov=False)
adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(adam, 'categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.samples // batch_size,
        epochs=40,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // batch_size,
        callbacks=[PlotLossesKeras()],
        verbose=1)